In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms, models
import timeit
import numpy as np
from tqdm.notebook import tqdm
import os
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Configurar os seeds para tentar manter a reprodutibilidade e uma comparação mais justa entre os treinamentos que realizarmos

In [2]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

g = torch.Generator()
g.manual_seed(0)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
    
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Device: {device}')

Device: cuda


Função para calcular valores médios e desvio padrão de um dataset com 3 canais

In [3]:
# Fonte: https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/03/08/image-mean-std.html
def calcule_mean_std(loader, image_size):
    start = timeit.time.perf_counter()

    # placeholders
    psum    = torch.tensor([0.0, 0.0, 0.0])
    psum_sq = torch.tensor([0.0, 0.0, 0.0])

    # loop through images
    nb_samples = 0
    for data in tqdm(loader):
        inputs, labels = data
        # [batch_size x 3 x image_size x image_size]
        psum    += inputs.sum(axis        = [0, 2, 3])
        psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])
        nb_samples += inputs.size(0)

    # pixel count
    count = nb_samples * image_size * image_size

    # mean and std
    total_mean = psum / count
    total_var  = (psum_sq / count) - (total_mean ** 2)
    total_std  = torch.sqrt(total_var)

    # output
    print('mean: '  + str(total_mean))
    print('std:  '  + str(total_std))
    print("Elapsed time: %.3f seconds" % (timeit.time.perf_counter() - start))

Função de treinamento

In [4]:
def big_fit(dataset_name, log_name='', image_size=224, total_epoch=40, MEAN=[0.485, 0.456, 0.406], STD=[0.229, 0.224, 0.225]):
    # Resnet18
    model = models.resnet18()  # 'IMAGENET1K_V2'
    num_ftrs = model.fc.in_features
    if dataset_name == 'cifar100':
        model.fc = nn.Linear(num_ftrs, 100)
    elif dataset_name == 'flowers102':
        model.fc = nn.Linear(num_ftrs, 102)
    else:
        raise ValueError('dataset_name not available!')
        
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    if MEAN != []:
        transform_default = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN, std=STD)
        ])

        transform_augmentation = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),

            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN, std=STD)
        ])
    else:
        transform_default = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.Lambda(lambda image: torch.tensor(np.array(image).astype(np.float32).reshape(3, 128, 128))) 
        ])

        transform_augmentation = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            
            transforms.Resize((image_size, image_size)),
            transforms.Lambda(lambda image: torch.tensor(np.array(image).astype(np.float32).reshape(3, 128, 128))) 
        ])

    if dataset_name == 'cifar100':
        dataset_train = torchvision.datasets.CIFAR100('data/cifar100', train=True, download=True, transform=transform_augmentation)
        dataset_test = torchvision.datasets.CIFAR100('data/cifar100', train=False, download=True, transform=transform_default)
    elif dataset_name == 'flowers102':
        dataset_train = torchvision.datasets.Flowers102('data/flowers102', 'train', download=True, transform=transform_augmentation)
        dataset_test = torchvision.datasets.Flowers102('data/flowers102', 'test', download=True, transform=transform_default)
    else:
        raise ValueError('dataset_name not available!')
        
    train_loader = torch.utils.data.DataLoader(
                        dataset_train, batch_size=128,
                        shuffle=True, num_workers=4,
                        worker_init_fn=seed_worker,
                        generator=g)

    test_loader = torch.utils.data.DataLoader(
                    dataset_test, batch_size=128,
                    shuffle=False, num_workers=4,
                    worker_init_fn=seed_worker,
                    generator=g)

    if log_name != '':
        writer = SummaryWriter(f'runs/{log_name}')  # 'runs/50'
    else:
        writer = SummaryWriter()
        
    results = []
    # acc, loss, epoch
    best_acc = [0., 0., 0]

    for epoch in range(total_epoch):
        torch.cuda.empty_cache()
        model.train()
        running_loss = 0.
        running_accuracy = 0.
        running_p = []
        running_y = []
        for data in tqdm(train_loader):
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Calcule accuracy
            p = outputs.cpu().detach()
            labels = labels.cpu().detach()
            top_p, top_class_p = p.topk(k = 1, dim = 1)
            top_class_l = labels
            running_accuracy += accuracy_score(top_class_l, top_class_p)
        writer.add_scalar(f'{dataset_name}-Accuracy/train', running_accuracy/len(train_loader), epoch)
        writer.add_scalar(f'{dataset_name}-Loss/train', running_loss/len(train_loader), epoch)
        print('Epoch {}; Loss {}; Accuracy {}'.format(epoch, running_loss/len(train_loader), running_accuracy/len(train_loader)))

        if epoch % 1 == 0:
            model.eval()
            running_accuracy_test = 0.
            running_loss_test = 0.
            running_p2 = []
            running_y2 = []
            for data in tqdm(test_loader):
                inputs_test, labels_test = data
                inputs_test = inputs_test.to(device)
                labels_test = labels_test.to(device)

                outputs_test = model(inputs_test)
                loss = criterion(outputs_test, labels_test)
                running_loss_test += loss.item()

                # Calcule accuracy
                p = outputs_test.cpu().detach()
                labels = labels_test.cpu().detach()
                top_p, top_class_p = p.topk(k = 1, dim = 1)
                top_class_l = labels
                running_accuracy_test += accuracy_score(top_class_l, top_class_p)
            writer.add_scalar(f'{dataset_name}-Accuracy/test', running_accuracy_test/len(test_loader), epoch)
            writer.add_scalar(f'{dataset_name}-Loss/test', running_loss_test/len(test_loader), epoch)
            print(f'Loss test {running_loss_test/len(test_loader)}; Accuracy test:{running_accuracy_test/len(test_loader)}')
            if running_accuracy_test/len(test_loader) > best_acc[0]:
                best_acc = [running_accuracy_test/len(test_loader), running_loss_test/len(test_loader), epoch]
        writer.flush()
    writer.flush()
    return best_acc

## Treinamento dos datasets

Os melhores resultados de cada treinamento serão armazenados no dicionário "all_best_result" com seus respectivos nomes, sendo eles:
- [dataset]-imagenet == Média e desvio padrão do dataset ImageNet
- [dataset]-custom == Média e desvio padrão calculados do dataset utilizado
- [dataset]-0-1 == Valores normalizados entre 0 e 1
- [dataset]--1-1 == Valores normalizados entre -1 e 1
- [dataset]-255 == Valores brutos das imagens, ou seja, entre 0 e 255

In [5]:
all_best_result = {}

### Cifar100

In [6]:
image_size = 128
total_epoch = 60

Calcular a média e devio padrão do dataset

In [7]:
transform_default = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=MEAN, std=STD)
])

dataset_train = torchvision.datasets.CIFAR100('data/cifar100', train=True, download=True, transform=transform_default)

train_loader = torch.utils.data.DataLoader(
                    dataset_train, batch_size=128,
                    shuffle=True, num_workers=4,
                    worker_init_fn=seed_worker,
                    generator=g)

calcule_mean_std(train_loader, image_size)

Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

mean: tensor([0.5075, 0.4870, 0.4413])
std:  tensor([0.2620, 0.2510, 0.2712])
Elapsed time: 5.428 seconds


In [8]:
all_best_result['cifar100-imagenet'] = big_fit(dataset_name='cifar100', log_name='imagenet', image_size=image_size, total_epoch=total_epoch)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 0; Loss 3.678847061703577; Accuracy 0.13419117647058823


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 3.309518623955642; Accuracy test:0.1895767405063291


...

Loss test 2.845173378533955; Accuracy test:0.5830696202531646


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 59; Loss 0.06981241853093095; Accuracy 0.9771379475703326


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 2.72951602030404; Accuracy test:0.599881329113924


In [9]:
all_best_result['cifar100-custom'] = big_fit(dataset_name='cifar100', log_name='custom', image_size=image_size, total_epoch=total_epoch, MEAN=[0.5075, 0.4870, 0.4413], STD=[0.2620, 0.2510, 0.2712])

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 0; Loss 3.6628949020219888; Accuracy 0.13622522378516624


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 3.374734615977806; Accuracy test:0.1841376582278481


...

Loss test 2.739788654484326; Accuracy test:0.5975079113924051


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 59; Loss 0.06937578570245363; Accuracy 0.976682384910486


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 2.7509186237673218; Accuracy test:0.5909810126582279


In [10]:
all_best_result['cifar100-0-1'] = big_fit(dataset_name='cifar100', log_name='0-1', image_size=image_size, total_epoch=total_epoch, MEAN=[0., 0., 0.], STD=[1, 1, 1])

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 0; Loss 3.701076189880176; Accuracy 0.12737771739130435


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 3.393786324730402; Accuracy test:0.1863132911392405


...

Loss test 2.748211979866028; Accuracy test:0.5921677215189873


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 59; Loss 0.06448986970216912; Accuracy 0.978656489769821


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 2.8392200092726116; Accuracy test:0.5874208860759493


In [11]:
all_best_result['cifar100--1-1'] = big_fit(dataset_name='cifar100', log_name='-1-1', image_size=image_size, total_epoch=total_epoch, MEAN=[0.5, 0.5, 0.5], STD=[0.5, 0.5, 0.5])

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 0; Loss 3.700665853822323; Accuracy 0.13075047953964195


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 3.49266405648823; Accuracy test:0.16020569620253164


...

Loss test 2.715272511108012; Accuracy test:0.5940466772151899


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 59; Loss 0.07495218916031558; Accuracy 0.9753276854219948


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 2.7418387630317786; Accuracy test:0.5863330696202531


In [12]:
all_best_result['cifar100-255'] = big_fit(dataset_name='cifar100', log_name='255', image_size=image_size, total_epoch=total_epoch, MEAN=[], STD=[])

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 0; Loss 4.037272183791451; Accuracy 0.08351982097186701


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 3.8484369923796833; Accuracy test:0.10690268987341772


...

Loss test 5.861796686920939; Accuracy test:0.2924248417721519


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 59; Loss 0.14643417661795227; Accuracy 0.9528692455242966


  0%|          | 0/79 [00:00<?, ?it/s]

Loss test 5.897826291337798; Accuracy test:0.29371044303797467


### Flowers102

In [13]:
image_size = 128
total_epoch = 150

Calcular a média e devio padrão do dataset

In [14]:
transform_default = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=MEAN, std=STD)
])

dataset_train = torchvision.datasets.Flowers102('data/flowers102', 'train', download=True, transform=transform_default)

train_loader = torch.utils.data.DataLoader(
                    dataset_train, batch_size=128,
                    shuffle=True, num_workers=4,
                    worker_init_fn=seed_worker,
                    generator=g)

calcule_mean_std(train_loader, image_size)

  0%|          | 0/8 [00:00<?, ?it/s]

mean: tensor([0.4330, 0.3819, 0.2964])
std:  tensor([0.2895, 0.2407, 0.2683])
Elapsed time: 1.212 seconds


In [15]:
all_best_result['flowers102-imagenet'] = big_fit(dataset_name='flowers102', log_name='imagenet', image_size=image_size, total_epoch=total_epoch)

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 0; Loss 4.5560925006866455; Accuracy 0.028540826612903226


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 8.939744871489856; Accuracy test:0.029974489795918366


  0%|          | 0/8 [00:00<?, ?it/s]

...

Loss test 3.8632950667215855; Accuracy test:0.39642857142857146


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 149; Loss 0.0005099356967548374; Accuracy 1.0


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 3.8663970596936283; Accuracy test:0.39754464285714286


In [16]:
all_best_result['flowers102-custom'] = big_fit(dataset_name='flowers102', log_name='custom', image_size=image_size, total_epoch=total_epoch, MEAN=[0.4330, 0.3819, 0.2964], STD=[0.2895, 0.2407, 0.2683])

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 0; Loss 4.542329728603363; Accuracy 0.031439012096774195


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 6.410482863990628; Accuracy test:0.018973214285714284


  0%|          | 0/8 [00:00<?, ?it/s]

...

Loss test 3.766424039796907; Accuracy test:0.39961734693877554


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 149; Loss 0.0005444859380077105; Accuracy 1.0


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 3.758815798224235; Accuracy test:0.3992984693877551


In [17]:
all_best_result['flowers102-0-1'] = big_fit(dataset_name='flowers102', log_name='0-1', image_size=image_size, total_epoch=total_epoch, MEAN=[0., 0., 0.], STD=[1, 1, 1])

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 0; Loss 4.5359490513801575; Accuracy 0.042212701612903226


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 5.651338061507867; Accuracy test:0.007334183673469388


  0%|          | 0/8 [00:00<?, ?it/s]

...

Loss test 3.7791678461493277; Accuracy test:0.3957908163265306


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 149; Loss 0.0006233570456970483; Accuracy 1.0


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 3.7849515852879505; Accuracy test:0.39626913265306124


In [18]:
all_best_result['flowers102--1-1'] = big_fit(dataset_name='flowers102', log_name='-1-1', image_size=image_size, total_epoch=total_epoch, MEAN=[0.5, 0.5, 0.5], STD=[0.5, 0.5, 0.5])

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 0; Loss 4.5578354597091675; Accuracy 0.035376764112903226


  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 5.548943806667717; Accuracy test:0.010841836734693877


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

Loss test 4.055930770173365; Accuracy test:0.3567283163265306


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 98; Loss 0.01841427496401593; Accuracy 0.99609375


  0%|          | 0/49 [00:00<?, ?it/s]

In [19]:
all_best_result['flowers102-255'] = big_fit(dataset_name='flowers102', log_name='255', image_size=image_size, total_epoch=total_epoch, MEAN=[], STD=[])

## Results

Os resultados estão separados por dataset e seus respectivos métodos de normalizações.

In [20]:
cifar100_best_results = {key: value for key, value in all_best_result.items() if key.startswith('cifar100')}
flowers102_best_results = {key: value for key, value in all_best_result.items() if key.startswith('flowers102')}

Os valores das listas internas estão ordenados por melhor acurácia. Sendo eles: [acurácia, loss, época]

In [21]:
sorted(cifar100_best_results.items(), key=lambda item: item[1][0], reverse=True)

[('cifar100--1-1', [0.603342592716217, 2.5810809135437, 53]),
 ('cifar100-custom', [0.602155864238739, 2.47704648971558, 41]),
 ('cifar100-imagenet', [0.59988135099411, 2.72951602935791, 59]),
 ('cifar100-0-1', [0.597804605960846, 2.60320425033569, 48]),
 ('cifar100-255', [0.311807751655579, 2.91237354278564, 17])]

In [22]:
sorted(flowers102_best_results.items(), key=lambda item: item[1][0], reverse=True)

[('flowers102-custom', [0.400255113840103, 3.74984192848206, 139]),
 ('flowers102-0-1', [0.397863507270813, 3.75747632980347, 140]),
 ('flowers102-imagenet', [0.397704094648361, 3.84556579589844, 138]),
 ('flowers102--1-1', [0.394355863332748, 3.80774807929993, 149]),
 ('flowers102-255', [0.206823974847794, 6.51196670532227, 106])]